In [30]:
import os
import numpy as np
import pandas as pd
from PyToF import ClassToF

X = ClassToF.ToF()

No reference radius supplied by the user. PyToF assumes R_ref = R_phys.


In [31]:
def calculate_MoI_numpy(radii, densities, method, total_mass=None):
    '''
    
    Parameters
    ----------
    radii : np.ndarray
        Array of radial distances from the center
    densities : np.ndarray
        Array of density values at corresponding radii
    method : str
        Integration method:
        - 'trapezoid': Trapezoidal rule (most accurate for discrete data)
        - 'simpson': Simpson's 1/3 rule (requires odd number of points)
    total_mass : float
        If provided, returns normalized moment of inertia (I/MR²)
    
    Returns
    -------
    moment_of_inertia : float
        The rotational moment of inertia I = (8π/3) ∫ ρ(r) r⁴ dr
    normalized_moi : float
        If total_mass provided, returns I/(M*R_max²)
    '''
    
    # Ensure arrays are numpy arrays
    radii = np.asarray(radii)
    densities = np.asarray(densities)
    
    if len(radii) != len(densities):
        raise ValueError('Radii and densities must have the same length')
    
    # Sort by radius
    sort_idx = np.argsort(radii)
    radii = radii[sort_idx]
    densities = densities[sort_idx]
    
    # Calculate the integrand: ρ(r) * r^4
    integrand = densities * radii**4
    
    # Perform integration based on method
    if method == 'trapezoid':
        # Trapezoidal rule: ∫f dx ≈ Σ (f_i + f_{i+1})/2 * Δr_i
        dr = np.diff(radii)
        integral = np.sum((integrand[:-1] + integrand[1:]) / 2 * dr)
        
    elif method == 'simpson':
        # Simpson's rule adapted for non-uniform spacing
        # Process consecutive triplets of points
        integral = 0.0
        i = 0
        
        while i < len(radii) - 2:
            # Take 3 points at a time
            r0, r1, r2 = radii[i:i+3]
            f0, f1, f2 = integrand[i:i+3]
            
            # Simpson's rule for non-uniform spacing
            h1 = r1 - r0
            h2 = r2 - r1
            
            # Coefficients for non-uniform Simpson's rule
            integral += (h1 + h2) / 6 * (
                f0 * (2*h1 - h2) / h1 +
                f1 * (h1 + h2)**2 / (h1 * h2) +
                f2 * (2*h2 - h1) / h2
            )
            
            i += 2  # Move by 2 points for next triplet
        
        # If there's a remaining point, use trapezoidal rule for the last interval
        if i == len(radii) - 2:
            dr = radii[-1] - radii[-2]
            integral += (integrand[-2] + integrand[-1]) / 2 * dr    
    else:
        raise ValueError(f'Unknown method: {method}')
    
    # Calculate moment of inertia: I = (8π/3) * integral
    moment_of_inertia = (8 * np.pi / 3) * integral
    
    # Calculate normalized moment of inertia if total mass is provided
    MoI = moment_of_inertia
    if total_mass is not None:
        R_max = radii[-1]
        MoI /= (total_mass * R_max**2)
    
    return MoI

def get_data_from_file(filename):
    folder_path = os.path.join('data', 'simulation_results', 'plot_MoI')
    file_path = os.path.join(folder_path, filename)

    df = pd.read_csv(file_path, names=['r', 'm', 'p', 'rho', 'I_norm', 'T'], skiprows=1)
    return df['r'].to_numpy(), df['rho'].to_numpy(), df['m'][0], df['I_norm'][0]

def sample_evenly(arr, max_len=1024):
    arr = np.asarray(arr)
    n = len(arr)
    if n <= max_len:
        return arr  # no sampling needed

    # choose 1024 indices evenly spaced from 0 to n-1
    idx = np.linspace(0, n - 1, max_len, dtype=int)
    return arr[idx]

def calculate_MoI_ToF(radii, densities, period):
    radii /= 100 # cm -> m
    densities *= 1000 # g/cm^3 -> kg/m^3

    mask = radii > 0.1*np.max(radii)
    radii = radii[mask]
    densities = densities[mask]
    
    # create smaller subsample 
    radii = sample_evenly(radii)
    densities = sample_evenly(densities)
    
    radius = max(radii)
    N = len(radii)

    mass = -4*np.pi*np.trapezoid(densities*radii**2, radii) # calculated mass in kg, negative sign because array starts with the outer surface
    X = ClassToF.ToF(N=N, M_phys=mass, R_phys=[radius, 'mean'], Period=period)

    X.li         = radii
    X.rhoi       = densities
    X.m_rot_calc = (2*np.pi/period)**2*X.li[0]**3/(X.opts['G']*mass)

    number_of_iterations = X.relax_to_shape()

    return X.get_NMoI(), number_of_iterations    

In [32]:
lit_MoI_dir = {
    'Earth'   : 0.3307, # Williams, James G. Contribution to the Earth's Obliquity Rate, Precession, and Nutation
    'Jupiter' : 0.254,  # [J]
    'Saturn'  : 0.210,  # [J]
    'Uranus'  : 0.225   # [J]
}

planet_files = {
    'Earth'   : 'Earth_04_analytical_MgSiO3_RK45_theta_2.csv',
    'Jupiter' : 'Jupiter_02_polytropic_RK45_theta_2.csv',
    'Saturn'  : 'Saturn_02_polytropic_RK45_theta_2.csv',
    'Uranus'  : 'Uranus_05_tabulated_H_RK45_theta_2.csv'
}

planet_periods = {
    'Earth'   : 86164,   # s
    'Jupiter' : 35730,   # s
    'Saturn'  : 38361.6, # s
    'Uranus'  : 62064    # s
}

In [33]:
for planet in ['Earth', 'Jupiter', 'Saturn', 'Uranus']:
    radii, densities, total_mass, MoI_shell = get_data_from_file(planet_files[planet])
    period = planet_periods[planet]

    MoI_trapezoid            = calculate_MoI_numpy(radii, densities, 'trapezoid', total_mass)
    MoI_simpson              = calculate_MoI_numpy(radii, densities, 'simpson', total_mass)
    MoI_lit                  = lit_MoI_dir[planet]
    MoI_ToF, num_itt         = calculate_MoI_ToF(radii, densities, period)
    MoI_ToF_inf, num_itt_inf = calculate_MoI_ToF(radii, densities, np.inf)

    print(f'=== {planet} ===')
    print(f'literature: {MoI_lit:.3f}')
    print(f'shell:      {MoI_shell:.3f}, diff: {(MoI_lit - MoI_shell):.3f}')
    print(f'trapezoid:  {MoI_trapezoid:.3f}, diff: {(MoI_lit - MoI_trapezoid):.3f}')
    print(f'simpson:    {MoI_simpson:.3f}, diff: {(MoI_lit - MoI_simpson):.3f}')
    print(f'ToF (inf):  {MoI_ToF_inf:.3f}, diff: {(MoI_lit - MoI_ToF_inf):.3f}, num itt: {num_itt_inf}')
    print(f'ToF:        {MoI_ToF:.3f}, diff: {(MoI_lit - MoI_ToF):.3f}, num itt: {num_itt}')
    print()


No reference radius supplied by the user. PyToF assumes R_ref = R_phys.
No reference radius supplied by the user. PyToF assumes R_ref = R_phys.
=== Earth ===
literature: 0.331
shell:      0.339, diff: -0.008
trapezoid:  0.342, diff: -0.011
simpson:    0.342, diff: -0.011
ToF (inf):  0.378, diff: -0.047, num itt: 1
ToF:        0.379, diff: -0.048, num itt: 45

No reference radius supplied by the user. PyToF assumes R_ref = R_phys.
No reference radius supplied by the user. PyToF assumes R_ref = R_phys.
=== Jupiter ===
literature: 0.254
shell:      0.273, diff: -0.019
trapezoid:  0.273, diff: -0.019
simpson:    0.273, diff: -0.019
ToF (inf):  0.273, diff: -0.019, num itt: 1
ToF:        0.283, diff: -0.029, num itt: 37

No reference radius supplied by the user. PyToF assumes R_ref = R_phys.
No reference radius supplied by the user. PyToF assumes R_ref = R_phys.
=== Saturn ===
literature: 0.210
shell:      0.204, diff: 0.006
trapezoid:  0.204, diff: 0.006
simpson:    0.204, diff: 0.006
ToF 